<a href="https://colab.research.google.com/github/elemnurguner/data-ai-projects/blob/main/Ak%C4%B1ll%C4%B1_%C3%9Cretim_Hatt%C4%B1_Denetim_Sistemi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Akıllı Üretim Hattı Denetim Sistemi 🏭🤖

**Bilgisayar Görüşü Tabanlı Otomatik Kalite Kontrol Sistemi**

## 📌 Proje Özeti
Bu proje, üretim hatlarındaki ürün kusurlarını gerçek zamanlı olarak tespit eden ve üretim hızını otomatik ayarlayan bir AI sistemidir. YOLOv8 segmentasyon modeli kullanılarak geliştirilmiş olup, endüstriyel otomasyon senaryolarına uygun şekilde tasarlanmıştır.

## ✨ Temel Özellikler
- 🎯 **Yüksek Hassasiyetli Tespit**: %70 güven eşiği ile yüzey kusurları ve şekil anomalileri
- ⚡ **Gerçek Zamanlı İşlem**: 1280x720 çözünürlükte 30 FPS işleme
- 🛠️ **PLC Entegrasyonu**: Simüle edilmiş üretim hattı kontrolü
- 🔄 **Adaptif Kontrol**: Kusur tespitinde otomatik hız ayarlama
- 📊 **Detaylı Raporlama**: Görsel analiz ve sistem durum raporları

## 🛠️ Teknoloji Stacki
| Bileşen | Teknoloji |
|---------|-----------|
| Computer Vision | OpenCV, YOLOv8 |
| AI Framework | PyTorch Lightning |
| Kamera Arayüzü | JavaScript + Colab |
| Veri Görselleştirme | Matplotlib |
| Simülasyon | OOP Python |

In [ ]:
# -*- coding: utf-8 -*-
"""
Akıllı Üretim Hattı Denetim Sistemi
Computer Vision + AI Agent for Quality Control
"""

# 1. Gerekli Kütüphaneler
!pip install opencv-python numpy matplotlib ultralytics pytorch-lightning torchvision
import cv2
import numpy as np
import time
from IPython.display import display, Javascript, HTML
from google.colab.output import eval_js
from base64 import b64decode
from ultralytics import YOLO
import matplotlib.pyplot as plt
import torch
import pytorch_lightning as pl

# 2. Özel Kalite Kontrol Modeli
class QualityControlModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = YOLO('yolov8n-seg.pt')  # Segmentasyon destekli model

    def detect_defects(self, img):
        results = self.model(img)
        defects = []

        for result in results:
            if result.boxes is not None:
                for box in result.boxes:
                    if box.conf > 0.7:  # %70 güven eşiği
                        x1, y1, x2, y2 = map(int, box.xyxy[0])
                        defects.append({
                            'bbox': [x1, y1, x2, y2],
                            'type': 'surface_defect' if box.cls == 0 else 'shape_anomaly',
                            'confidence': float(box.conf)
                        })
        return defects

# 3. PLC Simülasyonu
class PLC_Simulator:
    def __init__(self):
        self.production_line = {
            'speed': 100,
            'status': 'running',
            'error_codes': []
        }

    def send_command(self, cmd):
        if cmd == 'stop_line':
            self.production_line['status'] = 'stopped'
        elif cmd == 'slow_down':
            self.production_line['speed'] = max(30, self.production_line['speed']-20)
        return self.production_line.copy()

# 4. Geliştirilmiş Kamera Sistemi
def get_camera_feed():
    for attempt in range(3):
        try:
            js = Javascript('''
            async function capture() {
              const div = document.createElement('div');
              const video = document.createElement('video');
              const btn = document.createElement('button');
              btn.textContent = 'Kamera İzni Ver';

              video.style.display = 'block';
              const stream = await navigator.mediaDevices.getUserMedia({
                  video: {
                    width: {ideal: 1280},
                    height: {ideal: 720},
                    facingMode: 'environment'
                  }
              });

              div.appendChild(btn);
              div.appendChild(video);
              document.body.appendChild(div);
              video.srcObject = stream;
              await video.play();

              await new Promise(resolve => btn.onclick = resolve);

              const canvas = document.createElement('canvas');
              canvas.width = video.videoWidth;
              canvas.height = video.videoHeight;
              canvas.getContext('2d').drawImage(video, 0, 0);
              stream.getVideoTracks()[0].stop();
              div.remove();
              return canvas.toDataURL('image/jpeg', 0.9);
            }
            ''')
            display(js)
            return eval_js('capture()')
        except Exception as e:
            print(f"Deneme {attempt+1}/3: {str(e)}")
            time.sleep(1)

    # Kamera çalışmazsa test görseli
    print("Test görseli kullanılıyor...")
    !wget https://storage.googleapis.com/kaggle-datasets-images/1333910/2238853/cd3e07f6b9e359e565b6a9e8b1999c88/dataset.jpg -O test_product.jpg
    img = cv2.imread('test_product.jpg')
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# 5. Ana Kontrol Sistemi
def production_monitoring():
    print("⚙️ Akıllı Üretim Sistemi Başlatılıyor...")

    # Sistemleri başlat
    qc_model = QualityControlModel()
    plc = PLC_Simulator()

    try:
        # Görüntü al
        print("📷 Kamera bağlantısı kuruluyor...")
        image_data = get_camera_feed()

        if isinstance(image_data, str):  # Kamera görüntüsü ise
            img_bytes = b64decode(image_data.split(',')[1])
            img_array = np.frombuffer(img_bytes, dtype=np.uint8)
            frame = cv2.imdecode(img_array, flags=cv2.IMREAD_COLOR)
        else:  # Test görseli ise
            frame = image_data

        # Kalite kontrol
        print("🔍 Ürün analizi yapılıyor...")
        defects = qc_model.detect_defects(frame)

        # Otomasyon kararları
        if len(defects) > 0:
            print(f"⚠️ {len(defects)} hata tespit edildi!")
            plc_status = plc.send_command('slow_down')

            # Görselleştirme
            for defect in defects:
                x1, y1, x2, y2 = defect['bbox']
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(frame, f"{defect['type']} {defect['confidence']:.2f}",
                           (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                           (0, 0, 255), 2)
        else:
            print("✅ Ürün kalitesi uygun")
            plc_status = plc.send_command('speed_up')

        # Sonuçlar
        plt.figure(figsize=(15, 10))
        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        plt.title(f"Üretim Hattı - Durum: {plc_status['status'].upper()}")
        plt.axis('off')
        plt.show()

        print("\n📊 Sistem Raporu:")
        print(f"Üretim Hızı: {plc_status['speed']}%")
        print(f"Hat Durumu: {plc_status['status']}")
        if defects:
            print(f"Tespit Edilen Hatalar: {len(defects)} adet")

    except Exception as e:
        print(f"❌ Sistem hatası: {str(e)}")
        plc.send_command('stop_line')

# 6. Çalıştırma
if __name__ == "__main__":
    production_monitoring()